### Notebook to move cleaned ethograms to ground truth data set

In [17]:
import pandas as pd
from ipywidgets import HBox, VBox, Select
from fastplotlib import ImageWidget, Plot
from mesmerize_core.arrays import LazyVideo
from pathlib import Path
import numpy as np
from fastplotlib.graphics.selectors import LinearSelector
from fastplotlib.graphics import ImageGraphic
from tqdm import tqdm
import shutil
import os

In [2]:
ETHOGRAM_COLORS = {
    "lift": "b",
    "handopen": "green",
    "grab": "r",
    "sup": "cyan",
    "atmouth": "magenta",
    "chew": "yellow"
}

In [3]:
df = pd.read_hdf("/data/caitlin/cleaned_ethograms.hdf")
df

,trial,final_ethogram
0,M232_20170306_v074,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,M232_20170306_v024,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,M232_20170306_v025,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,M232_20170306_v059,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,M232_20170308_v028,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,M232_20170306_v076,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,M232_20170307_v064,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,M232_20170308_v034,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,M232_20170308_v070,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9,M232_20170314_v029,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [4]:
options = [k for k in list(df["trial"])]

In [5]:
len(options)

29

In [6]:
data_path = Path('/data/caitlin/potential_ground_truth/')

In [7]:
def trial_change(obj):
    selected_vid = data_path.joinpath(trial_selector.value).with_suffix('.avi')
    new_data = [LazyVideo(selected_vid)]
    # iw.gridplot[0,0].clear()
    # iw.gridplot[0,0].add_graphic(ImageGraphic(new_data[0][0]))
    iw.set_data(new_data=new_data, reset_vmin_vmax=True)
    plot.clear()
    make_plot(plot)
    plot.auto_scale()

In [8]:
trial_selector = Select(options=options)
trial_selector.observe(trial_change, "value")

In [9]:
iw = ImageWidget(data=[LazyVideo(data_path.joinpath(trial_selector.value).with_suffix('.avi'))])

RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


In [10]:
def ethogram_event_handler(ev):
    ix = ev.pick_info["selected_index"]
    iw.sliders["t"].value = ix

In [11]:
plot = Plot(size=(500, 100))
def make_plot(plot):
    
    ix = df.loc[df["trial"] == trial_selector.value].index[0]
    ethogram_array = df.loc[:,'final_ethogram'].loc[ix] 
    y_bottom = 0
    for i, b in enumerate(ETHOGRAM_COLORS.keys()):
        xs = np.arange(ethogram_array.shape[1], dtype=np.float32)
        ys = np.zeros(xs.size, dtype=np.float32)

        lg = plot.add_line(
            data=np.column_stack([xs, ys]),
            thickness=10,
            name=b
        )

        lg.colors = 0
        lg.colors[ethogram_array[i] == 1] = ETHOGRAM_COLORS[b]

        y_pos = (i * -10) - 1
        lg.position_y = y_pos

    ethogram_selector = LinearSelector(
                selection=0,
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                parent=lg,
                end_points=(y_bottom, y_pos),
            )

    plot.add_graphic(ethogram_selector)
    ethogram_selector.selection.add_event_handler(ethogram_event_handler)

RFBOutputContext()

In [12]:
make_plot(plot)
VBox([
    HBox([iw.show(), trial_selector]),
    plot.show()
])

### want to move corresponding videos of cleaned ethogram to ground truth dir

In [15]:
pot_loc = Path('/data/caitlin/potential_ground_truth/')
ground_loc = Path('/data/caitlin/ground_truth/')      

In [18]:
for trial in tqdm(list(df["trial"])):
    # check if video already copied:
    if os.path.exists(ground_loc.joinpath(trial).with_suffix('.avi')):
        continue
    else:
        shutil.copy(src=pot_loc.joinpath(trial).with_suffix('.avi'), dst=ground_loc.joinpath(trial).with_suffix('.avi'))

100%|████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 51671.54it/s]


### want to add to ground truth dataframe where ethograms are stored

In [19]:
ground_truth_df = pd.read_hdf('/data/caitlin/ground_truth.hdf')
ground_truth_df

,trial,ethogram
0,M234_20170328_v004,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,M234_20170328_v005,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,M234_20170331_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,M234_20170403_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,M234_20170412_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...
130,M328_20201130_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
131,M336_20210612_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
132,M336_20210612_v002,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
133,M336_20210613_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [27]:
# check if key exists
for trial, etho in tqdm(zip(list(df["trial"]), list(df["final_ethogram"]))):
    if trial in list(ground_truth_df["trial"]):
        continue
    else:
        ground_truth_df.loc[len(ground_truth_df.index)] = [trial, etho]
ground_truth_df.to_hdf('/data/caitlin/ground_truth.hdf', key='df')

29it [00:00, 68836.91it/s]
/tmp/ipykernel_3717547/3702716187.py:7: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['trial', 'ethogram'], dtype='object')]

  ground_truth_df.to_hdf('/data/caitlin/ground_truth.hdf', key='df')


In [28]:
ground_truth_df

,trial,ethogram
0,M234_20170328_v004,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,M234_20170328_v005,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,M234_20170331_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,M234_20170403_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,M234_20170412_v001,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...
159,M234_20170330_v038,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
160,M234_20170330_v043,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
161,M234_20170330_v047,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
162,M234_20170330_v058,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
